# Pydap

[Pydap](http://pydap.org/) [[github](https://github.com/pydap/pydap)]

## Installing PyDap

Pydap is on conda-forge, however the handlers and responses are only on PyPI, as well as server.

It seems necessary to install [Handlers](http://pydap.org/en/latest/handlers.html), e.g.

    pip install Pydap[handlers.netcdf]

Similarly there are [Responses](http://pydap.org/en/latest/responses.html) that may be installed, e.g.

    pip install pydap.responses.netcdf

Installing the netcdf response allows `.nc` to be appended.


In [ ]:
import os
from pprint import pprint

import rasterio as rio
import xarray as xr

## OPeNDAP Basics

- [QuickStart - OPeNDAP Documentation](https://opendap.github.io/documentation/QuickStart.html)
- [User Guide - OPeNDAP Documentation](https://opendap.github.io/documentation/UserGuideComprehensive.html)
- [The Hyrax Data Server Installation and Configuration Guide - OPeNDAP Documentation](https://opendap.github.io/hyrax_guide/Master_Hyrax_Guide.html)

Suffixes:

- `.info`---may be most useful!
- `.html`
- `.dds`
- `.das`
- ...
- .rdf?

DAP 4:

- `dsr`---dataset services, new and useful!
- `dmr`/`dmr.xml`

Returns in native OPeNDAP binary ecoding (`.dap`), NetCDF (`.nc`, `.nc4`), GeoTIFF (`.tiff`), JPEG2000, JSON (`.json`), and ASCII (`.ascii`). ???

Misc:

[Authentication For DAP Clients](https://opendap.github.io/hyrax_guide/Master_Hyrax_Guide.html#_authentication_for_dap_clients) discusses Earthdata, etc. An apparently older verion is at [DAP Clients - Authenticaion](https://docs.opendap.org/index.php/DAP_Clients_-_Authentication).

https://docs.opendap.org/index.php?title=DAP4_Specification

https://www.unidata.ucar.edu/software/thredds/v4.6/tds/tutorial/DAP.html --- DAP 4 has new/different suffixes, not comprehensive!

[Aggregation enhancements ](https://docs.opendap.org/index.php/Aggregation_enhancements) discusses [NcML](https://www.unidata.ucar.edu/software/thredds/current/netcdf-java/ncml/).

The relevance of the file extension on a DAP server is unclear!? 


## Pydap Basics


## Pydap Classes

Pydap implemements various classess to represent [The DAP data model](http://pydap.org/en/latest/developer.html#the-dap-data-model). The [Developer documentation](http://pydap.org/en/latest/developer.html) also sheds some light on things.

PyDap Types:

- `DatasetType`---a `StructureType`
- `BaseType`---data is scalar or ndarray
- `GridType`---strangely stored with 3 vhildren, the first is the array (BaseType) and the other two are axes
- `SequenceType`---includes `iterdata()` method
- `StructureType`---also behave like Python dict

PyDap types have attributes:

1. `name`
2. `id`
3. `attributes`---allow lazy access (with dot)
4. `data`

Some (which?) have:

- `dtype`
- `dimensions`
- ...

You'll generally get data through a variable's `data` attribute, but note that it may be necessary to slice it like `dataset.variable[:].data` for dap to handle it properly.

StructureTypes hold no data, but their `data` attribute collects data recursively from its children. Most Dap types seem to have a `children()` method but only seems relevant to StructureTypes. At least GridTypes have an `array` attribute for accessing BaseType array, so you may have to something like `dataset.variable.array[:].data`. 

Note that a dataset may contain a variable containing no actual data but with useful attributes!

Note that variables usually act like numpy arrays, incl. things like `shape()` method.


In [ ]:
from pydap.model import DatasetType, SequenceType, GridType, StructureType, BaseType, DapType
assert issubclass(BaseType, DapType)
assert issubclass(StructureType, DapType)
assert issubclass(DatasetType, StructureType)
assert issubclass(SequenceType, StructureType)
assert issubclass(GridType, StructureType)

## Using Pydap

[Using the client](http://pydap.org/en/latest/client.html)

Create `DatasetType` which is a fancy dict structure.

Get variables using dictionary syntax or attribute style access.

Variables can be `GridType`, ...?


In [ ]:
from pprint import pprint

In [ ]:
# A basic import is useless, it only gets you handlers, responses, and tests as empty namespaces!
# To do anything you probably need to import directly. Importing these namespaces will enable tab completion for their contents.
import pydap

In [ ]:
# Not sure this works, it should have to be installed separately.
# import pydap.handlers.netcdf

In [ ]:
# Pydap is bizarre, 
help(pydap)
help(pydap.handlers)
help(pydap.responses)
# help(pydap.tests)

In [ ]:
from pydap.client import open_url
url = 'http://www.ncei.noaa.gov/thredds/dodsC/namanl/201604/20160416/namanl_218_20160416_1800_000.grb'
dataset = open_url(url)  # Create DatasetType.

In [ ]:
pprint(dataset.name)
pprint(dataset.id)
pprint(dataset.attributes)
# pprint(dataset.data)  # Long output.

# list(dataset.keys())

# For fun, get types of the variables in the dataset.
# pprint([type(dataset[k]) for k, v in dataset.items()])

In [ ]:
dataset["Temperature_surface.Temperature_surface"]
dataset["Temperature_surface"]["Temperature_surface"]

## NASA Earthdata

Pydap used to support Earthdata auth using `pydap.utils.urs.install_basic_client()` (which seemed to read `.netrc`), but `pydap.utils` (which had to be separately installed) has gone away (no later than 1/2017). Now Earthdata auth is done using `pydap.cas.urs.setup_session()` to obtain a `requests.sessions.Session` object which is then passed to `open_url`.


The info (`.html`) page allows you to download as NetCDF 3 or 4, or as DAP 2 or 4.

The OPeNDAP directory listing ddx, dds, das, info, html, rdf, covjson, plus viewers:


Notes:

- The Dataset Viewers (viewers) leads you to:
  - DAP2
  - DAP4
  - w10n Service (Webification)---adds slash after file name!


Auth:

- [How To Access Data With PyDAP | Earthdata Wiki](https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+PyDAP) discusses using `pydap.util.urs` module but that's out of date! Other examples may work?
- [How To Access Data With Python](https://wiki.earthdata.nasa.gov/display/EL/How+To+Access+Data+With+Python) might work?


### SRTM Using OPeNDAP

NASA SRTM Version 3
[here](https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/contents.html)

I can't figure out how to use the `.ncml` files but thankfully there is a `netcdf` directory containing `.nc` files!

Relevant issues:

- https://github.com/pydap/pydap/issues/20 ---!
- https://github.com/pydap/pydap/issues/19
- https://github.com/pydap/pydap/pull/28 ---PR merged
- https://github.com/pydap/pydap/pull/11
- https://github.com/pydap/pydap/pull/26
- https://github.com/pydap/pydap/issues/51


#### Using Pydap

In [ ]:
# Read auth information from .netrc.
import netrc
auth = netrc.netrc()
login, account, password = auth.authenticators('urs.earthdata.nasa.gov')
# login, account, password

In [ ]:
from pydap.client import open_url        # Or import pydap.client to make available.
from pydap.cas.urs import setup_session  # Or import pydap.cas.urs to make available.

# Can't figure out how to work with nclm files, but thankfully SRTM is also available as netcdf.
dataset_url = 'https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc'

# Use login/password read from .netrc above.
session = setup_session(username=login, password=password, check_url=dataset_url)
dataset = open_url(dataset_url, session=session)
dataset

##### Explore Pydap Dataset

Dap objects contain the key attributes `name`, `id`, `attributes`, and `data`.

In [ ]:
print(dataset.name)
print(dataset.id)

In [ ]:
dataset.attributes

In [ ]:
dataset.data

While `data` in DatasetType recursively contains the data of all its children we can also list the immediate children. 

In [ ]:
list(dataset.keys())

In [ ]:
list(dataset.children())
# list(dataset.items())

In [ ]:
[v.attributes for k, v in dataset.items()]

In [ ]:
# For this SRTM dataset the lat and presumably lon variables seem redundant.
assert np.array_equal(dataset.lat[:].data, dataset.Band1.lat[:].data)

In [ ]:
# Here are a few other urls that use Earthdata login.

# ASTER DEM, works!
# dataset_url = 'https://opendap.cr.usgs.gov/opendap/hyrax/ASTGTM.002/ASTGTM2_N37E120_dem.nc'

# Another server, works!
# dataset_url = 'https://measures.gesdisc.eosdis.nasa.gov:443/opendap/LANDMET/LANDMET_ANC_SM.1/LANDMET_ANC_SM_L3_v1.nc'

In [ ]:
# # Explore...
# pydap.cas.urs.setup_session??

# # Explore...
# pydap.cas.urs.get_cookies.setup_session??

#### Using xarray

The ability to use authentication to open datasets in xarray using Pydap was added in PR [pydata/xarray#1570](https://github.com/pydata/xarray/pull/1570).

Interaction is similar to using Pydap directly. Pydap `attributes` are available in xarray `attr`. However it seems that the explicit call to `.data` is not necessary as it is with Pydap, which also avoids the weirdness of GridTypes.


In [ ]:
# Open authenticated datasets with xarray using pydap backend.
# Pattern from https://github.com/pydata/xarray/pull/1570.

import xarray as xr
import pydap.cas.urs  # Make setup_session() available.
import pydap.client   # Make open_url() available.

dataset_url = 'https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37W120.SRTMGL1.nc'

# Instantiate session.
session = pydap.cas.urs.setup_session(username=login, password=password, check_url=dataset_url)

# Method 1: More verbose.
# pydap_ds = pydap.client.open_url(dataset_url, session=session)
# store = xr.backends.PydapDataStore(pydap_ds)
# Method 2: More concise but equivalent.
store = xr.backends.PydapDataStore.open(dataset_url, session=session)
# Create xarray Dataset.
ds = xr.open_dataset(store)
ds

In this specific case we are working with SRTM data. 

In [ ]:
ds.coords

In [ ]:
ds.data_vars

In [ ]:
ds

In [ ]:
ds.crs.attrs

In [ ]:
crs = ds["crs"]

In [ ]:
# crs.values
pprint(crs.dims)
pprint(crs.coords)
pprint(crs.attrs)
pprint(crs.name)

---

In [ ]:
import rasterio as rio

In [ ]:
# tmp = rio.open(dataset_url, driver='DODS')

## Test local DEM

In [ ]:
lon = 'E120'
lat = 'N37'
url = 'zip+https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N37W120.SRTMGL1.hgt.zip!/N37W120.hgt'

In [ ]:
# hgt = rio.open(url, driver='SRTMHGT')

In [ ]:
# url = 'https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N37W120.SRTMGL1.hgt.zip'

In [ ]:
filename = os.path.expanduser('~/Downloads/N37W120.SRTMGL1.hgt.zip!/N37W120.hgt')

In [ ]:
dataset = rio.open('zip://' + filename, driver='SRTMHGT')
pprint(dataset.meta)
# pprint(hgt.profile)

In [ ]:
da = xr.open_rasterio(dataset)
da

In [ ]:
from rasterio.crs import CRS

In [ ]:
da.crs

In [ ]:
ds.crs

In [ ]:
dataset.crs.wkt

In [ ]:
repr(ds.crs.spatial_ref)

In [ ]:
print(ds.crs.spatial_ref)

In [ ]:
crs = ds.crs.spatial_ref.replace('\\', '')

CRS.from_wkt(crs)
# CRS.from_string(crs)
# CRS.from_user_input(crs)

In [ ]:
import re

In [ ]:
re.sub('\\', '', ds.crs.spatial_ref)

In [ ]:
rio.crs.CRS.from